Rapport du Projet de Déploiement en Conditions Réelles avec Docker

1. Introduction

Ce projet avait pour objectif de conteneuriser un modèle de machine learning existant et de le déployer via Docker, en exposant une API Flask pour effectuer des prédictions. Une partie importante du projet consistait à configurer un volume Docker pour stocker les requêtes et les prédictions, garantissant ainsi la persistance des données.

2. Étapes de Mise en Œuvre

2.1. Conteneurisation du Modèle

Téléchargement et préparation : Le modèle pré entraîné a été récupéré à partir d'un répertoire GitHub contenant un pipeline de prédiction.

Structure de l’application Flask : Création d’un fichier app.py contenant une API Flask permettant de recevoir des requêtes POST et de répondre avec des prédictions.

Dockerfile : Élaboration du fichier Docker pour construire l'image du projet :

In [ ]:
# Utilisez une image de base Python
FROM python:3.9-slim

# Définir le répertoire de travail dans le conteneur
WORKDIR /app

# Créez le volume pour /data
VOLUME /data

# Copier le dépôt GitHub cloné dans le répertoire de travail du conteneur
COPY . .

# Installer les dépendances
RUN pip install --no-cache-dir -r requirements.txt

# Expose le port pour l'API
EXPOSE 8000

# Commande pour démarrer l'application
CMD ["python", "app.py", "--host='0.0.0.0'","--port=8000"]

2.2. Configuration des Volumes

Création d’un volume Docker :

In [ ]:
docker volume create prediction_data

Montage du volume lors de l’exécution du conteneur :

In [ ]:
docker run -d -p 8000:8000 -v prediction_data:/data my-flask-app

Vérification de l'écriture des prédictions : Les prédictions et les données d’entrée sont écrites dans un fichier predictions.txt localisé dans le dossier monté /data.

2.3. Communication entre Conteneurs (Bonus)

Connexion à une base de données Dockerisée : Mise en place d’un conteneur de base de données (par exemple, PostgreSQL ou SQLite).

Tests de communication : Exécution de requêtes pour vérifier la bonne communication entre le conteneur Flask et la base de données.

3. Détails Techniques

Code pour l’écriture dans un volume :

In [ ]:
import os
@api.route("/predict", methods=["POST"])
def predict():
    input_data = request.json
    df = pd.DataFrame([input_data])
    prediction = model_pipeline.predict(df)[0]
    prediction_grade = prediction_mapping.get(prediction, prediction)
    if not os.path.exists("/data"):
        os.makedirs("/data")
    with open("/data/predictions.txt", "a") as f:
        f.write(f"Input: {input_data}, Prediction: {prediction_grade}\n")
    return jsonify({"prediction": prediction_grade})

4. Problèmes Rencontrés et Solutions

Erreur de port déjà utilisé : Correction en utilisant un autre port lors de l’exécution du conteneur (ex. : -p 5001:5000).

Absence de répertoire pour le fichier de prédictions : Ajout de la création automatique du répertoire /data dans le code.

5. Conclusion 

Ce projet a permis de déployer un modèle pré entraîné dans un environnement Docker, de configurer un volume pour sauvegarder les prédictions et de tester la communication avec une base de données. Cela renforce la compréhension du déploiement de services en conteneur et de la persistance des données dans un environnement isolé.

6. Perspectives Futures

Intégration d’un système de monitoring des prédictions.

Automatisation du déploiement avec Docker Compose pour la gestion des conteneurs multiples.